**DSCI 100 006 Group 38 Proposal**

**Title**: Predicting the likelihood of heart disease within a patient using 5 different factors.

**Introduction:**

The background information on the topic:
The dataset that we will be using for this project is one related to heart disease and the factors that contribute towards it. Different factors such as blood pressure, cholesterol and even age can have great impacts on the risk of contracting heart disease and so this data set provides past details of patients with attributes they had and whether or not they were healthy or not.

The question we will try to answer with our project:
The question that we will be addressing for our group project is, how do the age, sex, cholesterol, max heart rate, and resting blood pressure factors affect the likelihood of developing heart disease?

The dataset that we will use to answer the question:
Using a dataset found at this link, https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/, the data from the Cleveland dataset will be used. The original dataset contains 76 factors, however, the processed data, which is what will be used in our project, contains the following 14 factors:
Age, sex, chest pain type, resting blood pressure, cholesterol, fasting blood sugar, resting ECG, max heart rate, exercise-induced angina, oldpeak, slope, number of vessels coloured, thalassemia and the predicting class for if the patient is healthy or with heart-disease. We will further filter this data to be the 5 factors that we will be examining.

**Preliminary exploratory data analysis:**

Demonstrate that the dataset can be read from the web into R 
<br>Clean and wrangle your data into a tidy format
<br>Using only training data, summarize the data in at least one table (this is exploratory data analysis). 
 - An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 

<br>Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis).
- An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

In [3]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [8]:
heart_data<- read_csv(url("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"), col_names = FALSE) |>
    select(X1, X2, X4, X5, X8, X14)
colnames(heart_data) <- make.names(c("age", "sex", "restbps", "cholesterol", "maxbpm", "health"))
heart_data

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): X12, X13
dbl (12): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X14

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,sex,restbps,cholesterol,maxbpm,health
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
63,1,145,233,150,0
67,1,160,286,108,2
67,1,120,229,129,1
⋮,⋮,⋮,⋮,⋮,⋮
57,1,130,131,115,3
57,0,130,236,174,1
38,1,138,175,173,0


**Methods:**

Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
Describe at least one way that you will visualize the results

In order to conduct data analysis on the dataset we have retrieved we will make several visualisations to clearly see the relationship two variables have on each other, if it even exists. To be more specific we could make a scatterplot visualisation, with the cholesterol on the x-axis and blood pressure on the y-axis, and then colour the points based on whether the patient was deemed healthy (0) or sick (1, 2 or 3).

Another potential visualisation

Make scatter plots according to the factors:
Use the likelihood of heart disease as the vertical axis and ages(cholesterol, max heart rate, and resting blood pressure) in the horizontal axis, color the point by sex.
Find the relationship between the factors and likelihood of heart disease

**Expected outcomes and significance:**

What do you expect to find?
What impact could such findings have?
What future questions could this lead to?